In [ ]:
import tensorflow as tf
import numpy as np
from sys import getsizeof

In [ ]:
#===== CONSTANS =====
FEATURE_FILE = './data/features.npy'
LABEL_FILE = './data/labels.npy'
MODEL_NAME = 'resnet50'
PREPROCESSED_FEATURE_FILE = f'./data/{MODEL_NAME}_features.npy'
PREPROCESSED_LABEL_FILE = f'./data/{MODEL_NAME}_labels.npy'

In [ ]:
X = np.load(open(FEATURE_FILE, 'rb'))
Y = np.load(open(LABEL_FILE, 'rb'))

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

X = preprocess_input(X)
Y = Y.astype('float32').reshape((-1,1))

x_file = open(PREPROCESSED_FEATURE_FILE, 'wb')
y_file = open(PREPROCESSED_LABEL_FILE, 'wb')

np.save(x_file, X)
np.save(y_file, Y)

x_file.close()
y_file.close()

In [3]:
X = np.load(open(PREPROCESSED_FEATURE_FILE, 'rb'))
Y = np.load(open(PREPROCESSED_LABEL_FILE, 'rb'))

In [4]:
#Create class_weights for unbalanced dataset classes
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(class_weight = "balanced",
                                                  classes = np.unique(np.squeeze(Y)),
                                                  y = np.squeeze(Y))
class_weights = dict(enumerate(class_weights))

In [5]:
#Split into 5 folds evenly
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [6]:
#TESTING
class_weights

{0: 0.7518567059851463,
 1: 0.9566425792106726,
 2: 0.8948394644481996,
 3: 2.137888198757764,
 4: 0.8835836221280965,
 5: 0.7518567059851463,
 6: 1.7313883299798793}

In [7]:
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy 

#tf.compat.v1.disable_eager_execution()

for i, (train_index, test_index) in enumerate(skf.split(X, Y)):
    print(f"===== CURRENT FOLD: {i} =====")
    
    #PROOF OF CONCEPT (DELETE LATER)
    model = Sequential()
    resnet = ResNet50(include_top=False, 
                      input_shape=(224, 224, 3), 
                      weights='imagenet')

    model.add(resnet)
    model.add(Flatten())
    model.add(Dense(7, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    model.summary()

    model.fit(X[train_index], Y[train_index],
              class_weight=class_weights,
              epochs=1, 
              batch_size=4, 
              validation_split=0.2)
    
    tf.keras.backend.clear_session()
    del model

===== CURRENT FOLD: 0 =====
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 7)                 702471    
                                                                 
Total params: 24,290,183
Trainable params: 24,237,063
Non-trainable params: 53,120
_________________________________________________________________
2203/2203 [==============================] - 140s 58ms/step - loss: 2.1533 - accuracy: 0.3431 - val_loss: 2.6165 - val_accuracy: 0.0803
===== CURRENT FOLD: 1 =====
Model: "sequential"
_________________________________________________________________
 Layer (type)            

MemoryError: Unable to allocate 4.94 GiB for an array with shape (8811, 224, 224, 3) and data type float32

In [ ]:
tf.keras.backend.clear_session()
del model